In [1]:
import polars as pl
import datetime as dt

In [2]:
dt_from = dt.date(2020, 1, 1)
dt_to = dt.date(2030, 12, 31)

In [3]:
date_series = pl.date_range(
    dt_from,
    dt_to,
    '1d',
    eager=True
)

In [4]:
df = date_series.to_frame('date').lazy()

In [5]:
mapping_weekday_short = {
    1: 'Mo',
    2: 'Di',
    3: 'Mi',
    4: 'Do',
    5: 'Fr',
    6: 'Sa',
    7: 'So'
}
mapping_weekday_long = {
    1: 'Montag',
    2: 'Dienstag',
    3: 'Mittwoch',
    4: 'Donnerstag',
    5: 'Freitag',
    6: 'Samstag',
    7: 'Sonntag'
}
mapping_month_short = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mrz',
    4: 'Apr',
    5: 'Mai',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Okt',
    11: 'Nov',
    12: 'Dez'
}
mapping_month_long = {
    1: 'Januar',
    2: 'Februar',
    3: 'März',
    4: 'April',
    5: 'Mai',
    6: 'Juni',
    7: 'Juli',
    8: 'August',
    9: 'September',
    10: 'Oktober',
    11: 'November',
    12: 'Dezember'
}

In [10]:
df_date = (df
            .with_columns(
               pk_Datum = 'date',
               Datum_DIN = pl.col('date').dt.to_string('iso'),
               Tag_im_Monat = pl.col('date').dt.day(),
               Tag_im_Jahr = pl.col('date').dt.ordinal_day(),
               Wochentag_numerisch = pl.col('date').dt.weekday(),
               Wochentag_kurz = pl.col('date').dt.weekday().replace_strict(mapping_weekday_short),
               Wochentag_lang = pl.col('date').dt.weekday().replace_strict(mapping_weekday_long),
               Quartal_numerisch = pl.col('date').dt.quarter(),
               Quartal_lang = pl.col('date').dt.quarter().cast(pl.String) + '. Quartal',
               Quartal_kurz = 'Q' + pl.col('date').dt.quarter().cast(pl.String),
               Quartal_Jahr = pl.col('date').dt.year().cast(pl.String) + '-Q' + pl.col('date').dt.quarter().cast(pl.String),
               Quartal_Ende = pl.date(pl.col('date').dt.year(), pl.col('date').dt.quarter() * 3, 1).dt.month_end(), #end of quarter
               Quartal_Anfang = pl.date(pl.col('date').dt.year(), (pl.col('date').dt.quarter() * 3) - 2, 1).dt.month_start(), #begin of quarter
               Kalenderwoche_numerisch = pl.col('date').dt.week(),
               Kalenderwoche_kurz = 'KW ' + pl.col('date').dt.week().cast(pl.String).str.zfill(2),
               Kalenderwoche_Jahr = pl.col('date').dt.year().cast(pl.String) + ' | KW ' + pl.col('date').dt.week().cast(pl.String).str.zfill(2),
               Kalenderwoche_Anfang = pl.col('date').dt.offset_by('-' + ((pl.col('date').dt.weekday()) - 1).cast(pl.String) + 'd'),
               Kalenderwoche_Ende = pl.col('date').dt.offset_by((7 - (pl.col('date').dt.weekday())).cast(pl.String) + 'd'),
               Jahr = pl.col('date').dt.year(),
               Jahr_Anfang = pl.date(pl.col('date').dt.year(), 1, 1),
               Jahr_Ende = pl.date(pl.col('date').dt.year(), 12, 31),
               Monat_numerisch = pl.col('date').dt.month(),
               Monat_kurz = pl.col('date').dt.month().replace_strict(mapping_month_short),
               Monat_lang = pl.col('date').dt.month().replace_strict(mapping_month_long),
               Monat_Anfang = pl.date(pl.col('date').dt.year(), pl.col('date').dt.month(), 1),
            )
            .with_columns(
               Tag_im_Quartal = (pl.col('date') - pl.col('Quartal_Anfang')).dt.total_days() + 1, #day count in current quarter
               Monat_Jahr_kurz = pl.col('Monat_kurz') + ' ' + pl.col('Jahr').cast(pl.String),
               Monat_Jahr_lang = pl.col('Monat_lang') + ' ' + pl.col('Jahr').cast(pl.String),
               Monat_Jahr_numerisch = pl.col('Monat_numerisch').cast(pl.String).str.zfill(2) + '/' + pl.col('Jahr').cast(pl.String),
               Monat_Ende = pl.col('Monat_Anfang').dt.month_end(),               
            )
            .with_columns(                
                Tag_Folgetag = pl.col('date').dt.offset_by('1d'),
                Tag_Folgewoche = pl.col('date').dt.offset_by('1w'),
                Tag_Folgemonat = pl.col('date').dt.offset_by('1m'),
                Tag_FolgeJahr = pl.col('date').dt.offset_by('1y'),
                Tag_Vortag = pl.col('date').dt.offset_by('-1d'),
                Tag_Vorwoche = pl.col('date').dt.offset_by('-1w'),
                Tag_Vormonat = pl.col('date').dt.offset_by('-1m'),
                Tag_Vorjahr = pl.col('date').dt.offset_by('-1y'),
                Monat_Anzahl_Tage = pl.col('Monat_Ende').dt.day(),
                Heute = pl.lit(dt.datetime.today().date()),
                Ist_Diesen_Monat = (pl.col('date') >= pl.col('Monat_Anfang')) & (pl.col('date') <= pl.col('Monat_Ende')),
                Ist_Diese_Woche = (pl.col('date') >= pl.col('Kalenderwoche_Anfang')) & (pl.col('date') <= pl.col('Kalenderwoche_Ende')),
                Ist_Dieses_Quartal = (pl.col('date') >= pl.col('Quartal_Anfang')) & (pl.col('date') <= pl.col('Quartal_Ende')),
                Ist_Dieses_Jahr = (pl.col('date') >= pl.col('Jahr_Anfang')) & (pl.col('date') <= pl.col('Jahr_Ende'))
            )
            .with_columns(
                Ist_Heute = pl.col('date') == pl.col('Heute')                
            )
            .select(pl.exclude('date'))
           )

In [11]:
df_date.collect()

pk_Datum,Datum_DIN,Tag_im_Monat,Tag_im_Jahr,Wochentag_numerisch,Wochentag_kurz,Wochentag_lang,Quartal_numerisch,Quartal_lang,Quartal_kurz,Quartal_Jahr,Quartal_Ende,Quartal_Anfang,Kalenderwoche_numerisch,Kalenderwoche_kurz,Kalenderwoche_Jahr,Kalenderwoche_Anfang,Kalenderwoche_Ende,Jahr,Jahr_Anfang,Jahr_Ende,Monat_numerisch,Monat_kurz,Monat_lang,Monat_Anfang,Tag_im_Quartal,Monat_Jahr_kurz,Monat_Jahr_lang,Monat_Jahr_numerisch,Monat_Ende,Tag_Folgetag,Tag_Folgewoche,Tag_Folgemonat,Tag_FolgeJahr,Tag_Vortag,Tag_Vorwoche,Tag_Vormonat,Tag_Vorjahr,Monat_Anzahl_Tage,Heute,Ist_Diesen_Monat,Ist_Diese_Woche,Ist_Dieses_Quartal,Ist_Dieses_Jahr,Ist_Heute
date,str,i8,i16,i8,str,str,i8,str,str,str,date,date,i8,str,str,date,date,i32,date,date,i8,str,str,date,i64,str,str,str,date,date,date,date,date,date,date,date,date,i8,date,bool,bool,bool,bool,bool
2020-01-01,"""2020-01-01""",1,1,3,"""Mi""","""Mittwoch""",1,"""1. Quartal""","""Q1""","""2020-Q1""",2020-03-31,2020-01-01,1,"""KW 01""","""2020 | KW 01""",2019-12-30,2020-01-05,2020,2020-01-01,2020-12-31,1,"""Jan""","""Januar""",2020-01-01,1,"""Jan 2020""","""Januar 2020""","""01/2020""",2020-01-31,2020-01-02,2020-01-08,2020-01-01,2021-01-01,2019-12-31,2019-12-25,2019-12-31,2019-01-01,31,2025-05-04,true,true,true,true,false
2020-01-02,"""2020-01-02""",2,2,4,"""Do""","""Donnerstag""",1,"""1. Quartal""","""Q1""","""2020-Q1""",2020-03-31,2020-01-01,1,"""KW 01""","""2020 | KW 01""",2019-12-30,2020-01-05,2020,2020-01-01,2020-12-31,1,"""Jan""","""Januar""",2020-01-01,2,"""Jan 2020""","""Januar 2020""","""01/2020""",2020-01-31,2020-01-03,2020-01-09,2020-01-02,2021-01-02,2020-01-01,2019-12-26,2020-01-01,2019-01-02,31,2025-05-04,true,true,true,true,false
2020-01-03,"""2020-01-03""",3,3,5,"""Fr""","""Freitag""",1,"""1. Quartal""","""Q1""","""2020-Q1""",2020-03-31,2020-01-01,1,"""KW 01""","""2020 | KW 01""",2019-12-30,2020-01-05,2020,2020-01-01,2020-12-31,1,"""Jan""","""Januar""",2020-01-01,3,"""Jan 2020""","""Januar 2020""","""01/2020""",2020-01-31,2020-01-04,2020-01-10,2020-01-03,2021-01-03,2020-01-02,2019-12-27,2020-01-02,2019-01-03,31,2025-05-04,true,true,true,true,false
2020-01-04,"""2020-01-04""",4,4,6,"""Sa""","""Samstag""",1,"""1. Quartal""","""Q1""","""2020-Q1""",2020-03-31,2020-01-01,1,"""KW 01""","""2020 | KW 01""",2019-12-30,2020-01-05,2020,2020-01-01,2020-12-31,1,"""Jan""","""Januar""",2020-01-01,4,"""Jan 2020""","""Januar 2020""","""01/2020""",2020-01-31,2020-01-05,2020-01-11,2020-01-04,2021-01-04,2020-01-03,2019-12-28,2020-01-03,2019-01-04,31,2025-05-04,true,true,true,true,false
2020-01-05,"""2020-01-05""",5,5,7,"""So""","""Sonntag""",1,"""1. Quartal""","""Q1""","""2020-Q1""",2020-03-31,2020-01-01,1,"""KW 01""","""2020 | KW 01""",2019-12-30,2020-01-05,2020,2020-01-01,2020-12-31,1,"""Jan""","""Januar""",2020-01-01,5,"""Jan 2020""","""Januar 2020""","""01/2020""",2020-01-31,2020-01-06,2020-01-12,2020-01-05,2021-01-05,2020-01-04,2019-12-29,2020-01-04,2019-01-05,31,2025-05-04,true,true,true,true,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2030-12-27,"""2030-12-27""",27,361,5,"""Fr""","""Freitag""",4,"""4. Quartal""","""Q4""","""2030-Q4""",2030-12-31,2030-10-01,52,"""KW 52""","""2030 | KW 52""",2030-12-23,2030-12-29,2030,2030-01-01,2030-12-31,12,"""Dez""","""Dezember""",2030-12-01,88,"""Dez 2030""","""Dezember 2030""","""12/2030""",2030-12-31,2030-12-28,2031-01-03,2030-12-27,2031-12-27,2030-12-26,2030-12-20,2030-12-26,2029-12-27,31,2025-05-04,true,true,true,true,false
2030-12-28,"""2030-12-28""",28,362,6,"""Sa""","""Samstag""",4,"""4. Quartal""","""Q4""","""2030-Q4""",2030-12-31,2030-10-01,52,"""KW 52""","""2030 | KW 52""",2030-12-23,2030-12-29,2030,2030-01-01,2030-12-31,12,"""Dez""","""Dezember""",2030-12-01,89,"""Dez 2030""","""Dezember 2030""","""12/2030""",2030-12-31,2030-12-29,2031-01-04,2030-12-28,2031-12-28,2030-12-27,2030-12-21,2030-12-27,2029-12-28,31,2025-05-04,true,true,true,true,false
2030-12-29,"""2030-12-29""",29,363,7,"""So""","""Sonnt